## **Clasificacion binaria**

In [ ]:
# Imports de utilidades de Python
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

# Imports de Keras
from keras.datasets import mnist,fashion_mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils, plot_model
from keras.regularizers import l1

# let's keep our keras backend tensorflow quiet
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

In [ ]:
model = Sequential() 
model.add(Dense(1, activation='sigmoid')) #ultima capa tiene que ser asi
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer='adam')
#para problema de clasificacion binaria la ultima capa tiene que tener una unica neurona 
# con activacion sigmoide y la funcion de costo la elijo como binary cross-entropy


In [ ]:
history = model_simple.fit(X_train, Y_train,
          batch_size=32, epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test))

In [ ]:
salida = model_simple.predict(X_test[0:1])
# la salida es pedirle que prediga sobre algun ejemplo del test set 
print('Shape de la salida:',salida.shape)
print('Salida:',salida)

## **Optimizadores** 

El optimizador es el metodo que uso para moverme en el espacio de parametros para optimizar la funcion de costo.

In [ ]:
#metodo SGD
opti=tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='mse',optimizer=opti)

In [ ]:
# metodo momentum 
opti=tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
model.compile(loss='mse',optimizer=opti)

In [ ]:
#metodo adam
opti=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)
model.compile(loss='mse',optimizer=opti)

## **lmfit**
Si tengo datos y una forma funcional de mi sistema dinamico y quiero ajustar los valores de los parametros, puedo usar paquetes estadisticos como lmfit que tiene funciones como minimize. Esta funcion toma un objetivo y un objeto params y optimiza los valores de params para que el resultado se parezca lo mas posible a los datos posta. El objetivo es la funcion que quiero minimizar (el residuo en este caso). La busqueda de optimizacion de parametros se puede hacer por muchos medios que puedo elegir: fuerza bruta, Nelder-Mead, etc.

In [ ]:
trial = minimize(residual,params=params_inicial,args=(t, data),method=metodo, calc_covar=False)

El residual es la funcion por minimizar. La funcion minimize tiene como argumentos el objeto params, los argumentos (args son los datos necesarios para hacer el ajuste) y el metodo para realizar la busqueda de la optimizacion.

Encuentro los valores de los parametros que minimizan la funcion residuo. 

params se introducen aclarando el rango

In [ ]:
params_inicial = Parameters()
params_inicial.add('A', value=inicial[:,i][0],min=-bound,max=0)
params_inicial.add('B', value=inicial[:,i][1],min=-bound,max=0)
params_inicial.add('C', value=0,vary=False)
params_inicial.add('D', value=inicial[:,i][3],min=0,max=bound)

In [ ]:
def g(t, x0, paras): # solucion de la ODE con condicion inicial 'x0' y parametros 'paras'
    x = odeint(f, x0, t, args=(paras,))
    return x
def residual(paras, t, data): # funcion de costo a minimizar
    model = g(t,x0, paras)
    model = np.concatenate((model[:,0], model[:,2])) # los datos a contrastar son x y p, las componentes 0 y 2 del modelo
    return model-data

residual se define como la diferencia entre los datos y lo que prevee el flujo sobre los mismos tiempos en que se midio.

El campo vector necesario para calcular el flujo g, se declara previamente 

In [ ]:
x0 = [x[0],0,p[0]] # condicion inicial
data = np.concatenate((x,p)) # ajusto un solo vector que contiene las seniales de x y p

def f (z, tiempo, paras): # z=(x,y,p)
    A = paras['A'].value
    B = paras['B'].value
    C = paras['C'].value
    D = paras['D'].value

    x=z[0]
    y=z[1]
    p=z[2]
    # en principio permito que el campo vector tenga todos los terminos, pero despues puedo elegir desactivar E y H
  
    dxdt = y
    dydt = A*1 + B*x + C*y + D*p +E*funcion_forzante(tiempo)
    dpdt = E*x + F*y + G*p
    dzdt = [dxdt, dydt, dpdt]
    return dzdt


## **codigo posta en orden para correr de lmfit:**

In [ ]:
x0 = [x[0],0,p[0]] # condicion inicial
data = np.concatenate((x,p)) # ajusto un solo vector que contiene las seniales de x y p

def f (z, tiempo, paras): # z=(x,y,p)
    A = paras['A'].value
    B = paras['B'].value
    C = paras['C'].value
    D = paras['D'].value
    E = paras['E'].value
    F = paras['F'].value
    G = paras['G'].value
    H = paras['H'].value

    x=z[0]
    y=z[1]
    p=z[2]
    # en principio permito que el campo vector tenga todos los terminos, pero despues puedo elegir desactivar E y H
  
    dxdt = y
    dydt = A*1 + B*x + C*y + D*p +E*funcion_forzante(tiempo)  # no esta el termino Dp en lo de gabo
    dpdt = H*x + F*y + G*p # NO SE SI ES Ex o Hx . El termino Hx lo puedo volar, no esta en lo de gabo
    dzdt = [dxdt, dydt, dpdt]
    return dzdt

In [ ]:
def g(t, x0, paras): # solucion de la ODE con condicion inicial 'x0' y parametros 'paras'
    x = odeint(f, x0, t, args=(paras,))
    return x
def residual(paras, t, data): # funcion de costo a minimizar
    model = g(t,x0, paras)
    model = np.concatenate((model[:,0], model[:,2])) # los datos a contrastar son x y p, las componentes 0 y 2 del modelo
    return model-data

In [ ]:
params_inicial = Parameters()
params_inicial.add('A', value=inicial[:,i][0],min=-bound,max=0)
params_inicial.add('B', value=inicial[:,i][1],min=-bound,max=0)
params_inicial.add('C', value=inicial[:,i][2],min=-bound,max=0)
if p_on:
    params_inicial.add('D', value=inicial[:,i][3],min=-bound_p,max=0)
else:
    params_inicial.add('D', value=0,vary=False)  
params_inicial.add('E', value=inicial[:,i][4],min=0,max=bound)
params_inicial.add('F', value=inicial[:,i][5],min=0,max=bound)
params_inicial.add('G', value=inicial[:,i][6],min=-bound,max=0)
if x_on:
  

In [ ]:
trial = minimize(residual,params=params_inicial,args=(t, data),method=metodo, calc_covar=False)